In [9]:
from gym_maze.envs import BasicMultiAgent, AlternateMultiAgent, MazeEnv2, Adversary, AdvPro, AdvPro2, PAIRED
from gym_maze.envs.generators import RandomMazeGenerator
import matplotlib.pyplot as plt
import gym

import os
import pandas as pd
import shutil
import ray

from ray.tune.registry import register_env
import ray.rllib.agents.ppo as ppo

In [10]:
chkpt_root = "tmp/trial"

shutil.rmtree(chkpt_root, ignore_errors=True, onerror=None)
ray_results = "{}/ray_results/".format(os.getenv("HOME"))
shutil.rmtree(ray_results, ignore_errors=True, onerror=None)

ray.init(ignore_reinit_error=True, local_mode=True)

2021-05-17 11:44:39,171	INFO worker.py:665 -- Calling ray.init() again after it has already been called.


In [11]:
def run_one_episode_adv(env):
    '''Given a gym environment (env) it computer one episode and return cumulative reward.
    Used to check if our gym environments work'''
    
    env.reset()
    sum_reward = 0
    
    steps = []
    frames = []
    
    steps.append(env.state.copy())


    while not env.done:
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        steps.append(env.state.copy())
        sum_reward += reward
        
    return env

# AdvPro2

In [4]:
adversary = Adversary(n_clutter=10, size=8)
adversary = run_one_episode_adv(adversary)
protagonist = MazeEnv2(adversary.state['image'])
adversary = Adversary(n_clutter=10, size=8)

policies = {
        "ppo_policy_0": (ppo.PPOTFPolicy, adversary.observation_space, adversary.action_space, {}),
        "ppo_policy_1": (ppo.PPOTFPolicy, protagonist.observation_space, protagonist.action_space, {})
    }

def policy_mapping_fn(agent_id):
    if agent_id == 0:
        return "ppo_policy_0"
    else:
        return "ppo_policy_1"

In [5]:
select_env = 'AdvPro-v1'
env_config = AdvPro2(adversary, protagonist, save=True, file_name='advpro2_10_8/', MAX_STEPS=300)
register_env(select_env, lambda config: env_config)

In [6]:
agent = ppo.PPOTrainer(
        env=select_env,
        config={
            "multiagent": {
                "policies": policies,
                "policy_mapping_fn": policy_mapping_fn,
                "policies_to_train": ["ppo_policy_0", "ppo_policy_1"],
            },
            "vf_loss_coeff": 0.01,
            "framework": "tf",
        })

2021-05-17 11:42:01,738	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-05-17 11:42:01,739	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-05-17 11:42:01,824	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2021-05-17 11:42:02,897	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-17 11:42:04,026	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-17 11:42:04,891	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-17 11:42:05,903	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-17 11:42:06,671	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-17 11:42:11,824	INFO trainable.py:103 -- Trainable.setup took 10.088 sec

In [7]:
df = pd.DataFrame(columns = ['num','adv_min','adv_mean','adv_max', \
                            'pro_min','pro_mean','pro_max'], dtype=float)

In [8]:
status = "{:2d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:4.2f} saved {}"
n_iter = 1000
for n in range(n_iter):
    result = agent.train()
    chkpt_file = agent.save(chkpt_root)
    min_reward = result['policy_reward_min']
    mean_reward  = result['policy_reward_mean']
    max_reward = result['policy_reward_max']
    id0 = "ppo_policy_0"
    id1 = "ppo_policy_1"
    #print(n + 1, 'Agent -  min - mean - max ')
    #print('-----------------------------------------------------')
    #print('Adv ', round(min_reward[id0], 2), '  ', round(mean_reward[id0], 2), '  ', round(max_reward[id0], 2))
    print('Pro ', round(min_reward[id1], 2), '  ', round(mean_reward[id1], 2), '  ', round(max_reward[id1], 2))
    #print(' ')
    df = df.append({'num': n+1,'adv_min': min_reward[id0],'adv_mean': mean_reward[id0],'adv_max': max_reward[id0], \
                            'pro_min': min_reward[id1],'pro_mean': mean_reward[id1],'pro_max': max_reward[id1]}, ignore_index=True)

Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
Pro  -120.81    -44.93    0.97
Pro  -120.81    -50.66    0.97


RayTaskError: [36mray::RolloutWorker.par_iter_next()[39m (pid=4420, ip=128.179.145.215)
  File "python/ray/_raylet.pyx", line 432, in ray._raylet.execute_task.function_executor
  File "/home/psanchezlopez/anaconda3/envs/animalai/lib/python3.7/site-packages/ray/util/iter.py", line 1152, in par_iter_next
    return next(self.local_it)
  File "/home/psanchezlopez/anaconda3/envs/animalai/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 327, in gen_rollouts
    yield self.sample()
  File "/home/psanchezlopez/anaconda3/envs/animalai/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 662, in sample
    batches = [self.input_reader.next()]
  File "/home/psanchezlopez/anaconda3/envs/animalai/lib/python3.7/site-packages/ray/rllib/evaluation/sampler.py", line 95, in next
    batches = [self.get_data()]
  File "/home/psanchezlopez/anaconda3/envs/animalai/lib/python3.7/site-packages/ray/rllib/evaluation/sampler.py", line 224, in get_data
    item = 

# PAIRED

In [4]:
adversary = Adversary(n_clutter=10, size=8)
adversary = run_one_episode_adv(adversary)
protagonist = MazeEnv2(adversary.state['image'])
antagonist = MazeEnv2(adversary.state['image'])
adversary = Adversary(n_clutter=10, size=8)

policies = {
        "ppo_policy_0": (ppo.PPOTFPolicy, adversary.observation_space, adversary.action_space, {}),
        "ppo_policy_1": (ppo.PPOTFPolicy, protagonist.observation_space, protagonist.action_space, {}),
        "ppo_policy_2": (ppo.PPOTFPolicy, antagonist.observation_space, antagonist.action_space, {})
    }

def policy_mapping_fn(agent_id):
    if agent_id == 0:
        return "ppo_policy_0"
    elif agent_id == 1:
        return "ppo_policy_1"
    elif agent_id == 2:
        return "ppo_policy_2"

In [5]:
select_env = 'Paired-v0'
env_config = PAIRED(adversary, protagonist, antagonist, save=True, \
                    file_name='paired_pos_10_8/', MAX_STEPS=300)
register_env(select_env, lambda config: env_config)

In [6]:
agent = ppo.PPOTrainer(
        env=select_env,
        config={
            "multiagent": {
                "policies": policies,
                "policy_mapping_fn": policy_mapping_fn,
                "policies_to_train": ["ppo_policy_0", "ppo_policy_1", "ppo_policy_2"],
            },
            "vf_loss_coeff": 0.01,
            "framework": "tf",
        })

2021-05-13 11:39:24,386	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-05-13 11:39:24,387	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-05-13 11:39:24,463	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


2021-05-13 11:39:25,311	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-13 11:39:26,433	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-13 11:39:27,885	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-13 11:39:28,627	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-13 11:39:29,592	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
2021-05-13 11:39:30,844	WARNING deprecation.py:34 -- DeprecationWarning: `frames

## 1. Uncoupled PAIRED: reward of the protagonist and antagonist independent of reward of adversary (shortest path length)

In [7]:
df = pd.DataFrame(columns = ['num','adv_min','adv_mean','adv_max', \
                            'pro_min','pro_mean','pro_max', \
                            'ant_min','ant_mean','ant_max'], dtype=float)


In [8]:
status = "{:2d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:4.2f} saved {}"
n_iter = 60
for n in range(n_iter):
    result = agent.train()
    chkpt_file = agent.save(chkpt_root)
    min_reward = result['policy_reward_min']
    mean_reward  = result['policy_reward_mean']
    max_reward = result['policy_reward_max']
    id0 = "ppo_policy_0"
    id1 = "ppo_policy_1"
    id2 = "ppo_policy_2"
    print(n + 1, 'Agent -  min - mean - max ')
    print('-----------------------------------------------------')
    print('Adv ', round(min_reward[id0], 2), '  ', round(mean_reward[id0], 2), '  ', round(max_reward[id0], 2))
    print('Pro ', round(min_reward[id1], 2), '  ', round(mean_reward[id1], 2), '  ', round(max_reward[id1], 2))
    print('Ant ', round(min_reward[id2], 2), '  ', round(mean_reward[id2], 2), '  ', round(max_reward[id2], 2))
    print(' ')
    
    df = df.append({'num': n+1,'adv_min': min_reward[id0],'adv_mean': mean_reward[id0],'adv_max': max_reward[id0], \
                            'pro_min': min_reward[id1],'pro_mean': mean_reward[id1],'pro_max': max_reward[id1], \
                            'ant_min': min_reward[id2],'ant_mean': mean_reward[id2],'ant_max': max_reward[id2]}, ignore_index=True)

Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
1 Agent -  min - mean - max 
-----------------------------------------------------
Adv  -2.0    3.29    7.0
Pro  -139.64    -34.79    -0.08
Ant  -79.91    -28.64    0.91
 
2 Agent -  min - mean - max 
-----------------------------------------------------
Adv  -42.0    -0.73    7.0
Pro  -500.0    -56.84    0.97
Ant  -500.0    -66.0    0.95
 
3 Agent -  min - mean - max 
-----------------------------------------------------
Adv  -42.0    0.08    7.0
Pro  -500.0    -64.92    0.97
Ant  -500.0    -65.97    0.95
 
4 Agent -  min - mean - max 
-----------------------------------------------------
Adv  -42.0    1.15    7.0
Pro  -500.0    -64.91    1.0
Ant  -500.0    -58.42    1.0
 
5 Agent -  min - mean - max 
-----------------------------------------------------
Adv  -42.0    1.26    7.0
Pro  -500.0    -74.46    1.0
Ant  -500.0    -61.0    1.0
 
6 Agent -  min - mean - max 
--------------------------------

## 2. Positive PAIRED: reward adversary sum of reward of the protagonist and antagonist

In [12]:
df = pd.DataFrame(columns = ['num','adv_min','adv_mean','adv_max', \
                            'pro_min','pro_mean','pro_max', \
                            'ant_min','ant_mean','ant_max'], dtype=float)

In [13]:
status = "{:2d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:4.2f} saved {}"
n_iter = 500
for n in range(n_iter):
    result = agent.train()
    chkpt_file = agent.save(chkpt_root)
    min_reward = result['policy_reward_min']
    mean_reward  = result['policy_reward_mean']
    max_reward = result['policy_reward_max']
    id0 = "ppo_policy_0"
    id1 = "ppo_policy_1"
    id2 = "ppo_policy_2"
    print(n + 1, 'Agent -  min - mean - max ')
    print('-----------------------------------------------------')
    print('Adv ', round(min_reward[id0], 2), '  ', round(mean_reward[id0], 2), '  ', round(max_reward[id0], 2))
    print('Pro ', round(min_reward[id1], 2), '  ', round(mean_reward[id1], 2), '  ', round(max_reward[id1], 2))
    print('Ant ', round(min_reward[id2], 2), '  ', round(mean_reward[id2], 2), '  ', round(max_reward[id2], 2))
    print(' ')
    
    df = df.append({'num': n+1,'adv_min': min_reward[id0],'adv_mean': mean_reward[id0],'adv_max': max_reward[id0], \
                            'pro_min': min_reward[id1],'pro_mean': mean_reward[id1],'pro_max': max_reward[id1], \
                            'ant_min': min_reward[id2],'ant_mean': mean_reward[id2],'ant_max': max_reward[id2]}, ignore_index=True)

StopIteration: 

In [9]:
status = "{:2d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:4.2f} saved {}"
n_iter = 500
for n in range(n_iter):
    result = agent.train()
    chkpt_file = agent.save(chkpt_root)
    min_reward = result['policy_reward_min']
    mean_reward  = result['policy_reward_mean']
    max_reward = result['policy_reward_max']
    id0 = "ppo_policy_0"
    id1 = "ppo_policy_1"
    id2 = "ppo_policy_2"
    print(n + 1, 'Agent -  min - mean - max ')
    print('-----------------------------------------------------')
    print('Adv ', round(min_reward[id0], 2), '  ', round(mean_reward[id0], 2), '  ', round(max_reward[id0], 2))
    print('Pro ', round(min_reward[id1], 2), '  ', round(mean_reward[id1], 2), '  ', round(max_reward[id1], 2))
    print('Ant ', round(min_reward[id2], 2), '  ', round(mean_reward[id2], 2), '  ', round(max_reward[id2], 2))
    print(' ')
    
    df = df.append({'num': n+1,'adv_min': min_reward[id0],'adv_mean': mean_reward[id0],'adv_max': max_reward[id0], \
                            'pro_min': min_reward[id1],'pro_mean': mean_reward[id1],'pro_max': max_reward[id1], \
                            'ant_min': min_reward[id2],'ant_mean': mean_reward[id2],'ant_max': max_reward[id2]}, ignore_index=True)

1 Agent -  min - mean - max 
-----------------------------------------------------
Adv  -596.04    -247.42    2.0
Pro  -299.01    -105.96    1.0
Ant  -299.01    -141.46    1.0
 
2 Agent -  min - mean - max 
-----------------------------------------------------
Adv  -596.04    -244.02    2.0
Pro  -299.01    -110.55    1.0
Ant  -299.01    -133.47    1.0
 
3 Agent -  min - mean - max 
-----------------------------------------------------
Adv  -596.04    -239.08    2.0
Pro  -299.01    -106.71    1.0
Ant  -299.01    -132.37    1.0
 
4 Agent -  min - mean - max 
-----------------------------------------------------
Adv  -596.04    -232.0    2.0
Pro  -299.01    -106.22    1.0
Ant  -299.01    -125.78    1.0
 
5 Agent -  min - mean - max 
-----------------------------------------------------
Adv  -596.04    -231.18    2.0
Pro  -299.01    -105.27    1.0
Ant  -299.01    -125.91    1.0
 
6 Agent -  min - mean - max 
-----------------------------------------------------
Adv  -596.04    -228.94    2

In [10]:
df.to_pickle("my_pickle.pkl")